# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# The query checks current working directory
print(os.getcwd())

# The query gets the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# A for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# The query joins the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(filepath,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
print(len(file_path_list))

30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
# We read the csv file, extract and append the records to the empty list created
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# The query import the required cassandra Cluster and
# esterblish the connection to the Cassandra instance in the local machine address: (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# The following query create KEYSPACE called cassudacity with 
# SimpleStrategy replication class and 1 replication factor
try:
    session.execute(""" CREATE KEYSPACE cassudacity WITH 
                    REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1 }
                    """)
except Exception as e:               
    print(e)

#### Set Keyspace

In [8]:
# query below connects the current session to the KEYSPACE cassudacity created 
try:
    session.set_keyspace('cassudacity')
except Exception as e:                       
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## song_detail_by_session table is created to model the music app history for query 1 stated below
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## the table is configured so that the records are uniquely partioned and identified by sessionId with itemInsession clustering key

query = """ CREATE TABLE IF NOT EXISTS song_detail_by_session (sessionId int, itemInSession int, artist text, firstName text,
           gender text, lastName text,length float, level text, location text, song text, userId int, PRIMARY KEY(sessionId,itemInSession))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [10]:
# The code reads the required records into file
# Extract the records without the header
# Inserted records are specified by indexing each column for every record extracted
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_detail_by_session ( sessionId, itemInSession, artist, firstName, gender, lastName, length,\
        level, location, song, userId)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], float((line[5])), line[6],
                                line[7], line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [22]:
## The query returns artist, song and length of the song from song_detail_by_session table 
## where sessionId = 338 and itemInSession = 4
print("artist | song | song_length")
query = "SELECT artist, song, length FROM song_detail_by_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows= session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

artist | song | song_length
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### REPEATING THE ABOVE THREE CELLS TO ADDRESS OTHER BUSINESS QUESTIONS

In [14]:
## song_detail_by_userid table is created to model the music app history for query 2 stated below
## name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182¶
## the table is configured so that the records are uniquely partioned and identified by both (userId,sessionid) 
## the song detail is ordered by itemInsession the clustering key

query = """ CREATE TABLE IF NOT EXISTS song_detail_by_userid (userId int,sessionId int,itemInSession int, artist text, firstName text, 
            gender text, lastName text, length float, level text, location text, song text, PRIMARY KEY((userId,sessionid),itemInSession))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
# The code reads the required records into file
# Extract the records without the header
# Inserted records are specified by indexing each column for every record extracted

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_detail_by_userid (userId, sessionId,  itemInSession, artist, firstName, gender, lastName, length,\
        level, location, song)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], float((line[5])), line[6],
                                line[7], line[9]))

In [20]:
## The query returns artist, song, firstname, and lastname from song_detail_by_userid WHERE userid = 10 and sessionid = 182
print("artist | song | firstName | lastName |")
query = "SELECT artist, song, firstName, lastName FROM song_detail_by_userid WHERE userid = 10 and sessionid = 182"
try:
    rows= session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

artist | song | firstName | lastName |
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [17]:
## usernames_by_song_name table is created to model the music app history for query 3 stated below
## firstname and lastname in the music app history who listened to the song 'All Hands Against His Own'
## the table is configured so that the records are uniquely partioned and clustered by song and userId respectively

query = """ CREATE TABLE IF NOT EXISTS usernames_by_song_name (song text, userId int, artist text, firstName text, gender text, 
           itemInSession int, lastName text, length float, level text, location text, sessionId int, PRIMARY KEY(song,userId))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [18]:
# The code reads the required records into file
# Extract the records without the header
# Inserted records are specified by indexing each column for every record extracted

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO usernames_by_song_name (song, userId, artist, firstName, gender, itemInSession, lastName, length,\
        level, location, sessionId)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[0], line[1], line[2], int(line[3]), line[4], float((line[5])), line[6],
                                line[7], int(line[8])))

In [21]:
## The query returns firstname and lastname of users who listened to song 'All Hands Against His Own'
print("firstName | lastName")
query = "SELECT firstName, lastName FROM usernames_by_song_name WHERE song='All Hands Against His Own'"
try:
    rows= session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

firstName | lastName
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [19]:
## The following queries DROP the three tables created in this session

try:
    session.execute("DROP TABLE song_detail_by_session")    ##
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE song_detail_by_userid")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE usernames_by_song_name")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()